This notebook describes the methods use to benchmark stereomapper performance on a set of diastereomeric pairs. This dataset was generated according to the methods described in the following [notebook](enantiomer_diastereomer_control.ipynb).

In [6]:
import os
import pandas as pd
from pathlib import Path
import sqlite3
import json
import subprocess
import re

In [ ]:
# run stereomapper as a subprocess
deduped_pairs_dir = Path.home('diastereomer_benchmark_data')
results = Path.home('diastereomer_benchmark_results.sqlite') 
cache_path = Path.home('diastereomer_benchmark_cache.sqlite') 

cmd =[
    "stereomapper",
    "run",
    "-d", deduped_pairs_dir.as_posix(),
    "-o", results.as_posix(),
    "-p", cache_path.as_posix(),
    "--fresh-cache"
]

subprocess.run(cmd) 

INFO    Logging initialised. File: /media/JACK/stereomapper_logs/stereomapper_20251104_121315.log
Pipeline: 100%|██████████████████████████████████████████████████████████| , Complete! [00:33<00:00]


✅ Pipeline completed in 33.1s
📦 Inputs attempted: 6,192 (skipped 0)
📊 Successes: 6,192 | Failures: 0
🔗 Inchikey groups — processed 3,071, skipped 0, failed 0
🧮 Relationship rows: 3,187
🧾 Unique inchikeys observed: 3,071
💾 Cache hit rate: 0.0%


Pipeline: 100%|██████████████████████████████████████████████████████████| , Complete! [00:33<00:00]


CompletedProcess(args=['stereomapper', 'run', '-d', '/home/jackmcgoldrick/drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/data/stereomapper/control_sets/stereoisomer_control/diastereomer_benchmark_data', '-o', '/home/jackmcgoldrick/drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/results/stereomapper/project_results/benchmarking/041125_diastereomer_benchmark_v4.sqlite', '-p', '/home/jackmcgoldrick/drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/results/stereomapper/project_results/benchmarking/cache/041125_diastereomer_benchmark.sqlite', '--fresh-cache'], returncode=0)

In [9]:
conn = sqlite3.connect(results)

moldf = pd.read_sql_query("SELECT * FROM relationships", conn)
conn.close()
moldf

,cluster_a,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
0,1,2,"[""chebi:CHEBI:224674""]","[""chebi:CHEBI:219347""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
1,3,4,"[""chebi:CHEBI:3829""]","[""chebi:CHEBI:81144""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
2,5,6,"[""chebi:CHEBI:69288""]","[""chebi:CHEBI:69289""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3,7,8,"[""chebi:CHEBI:117326""]","[""chebi:CHEBI:117283""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
4,9,10,"[""chebi:CHEBI:81326""]","[""chebi:CHEBI:81325""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
...,...,...,...,...,...,...,...,...,...,...,...
3182,6178,6179,"[""chebi:CHEBI:122230""]","[""chebi:CHEBI:122370""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3183,6180,6181,"[""chebi:CHEBI:188987""]","[""chebi:CHEBI:188993""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3184,6182,6183,"[""chebi:CHEBI:217768""]","[""chebi:CHEBI:208414""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3185,6184,6185,"[""chebi:CHEBI:197905""]","[""chebi:CHEBI:199614""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0


3168 rows, only expected 3097. This is due to off target relationships being computed on structure pairs which are related to one another. This is unavoidable in some cases, and will be solved using a pairkey match back on to the original dataframe to ensure only those control relationships are analysed.

In [4]:
 # read in the csv file
test_file = Path.home() / "drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/data/molDistance/control_sets/stereoisomer_control/diast_control_set_filtered.csv"

df_test_data = pd.read_csv(test_file)
head = df_test_data.head()
tail = df_test_data.tail()
print(head)
print(tail)

            id1           id2          label
0  CHEBI_122389  CHEBI_122317  Diastereomers
1  CHEBI_216464  CHEBI_215732  Diastereomers
2  CHEBI_120133  CHEBI_120171  Diastereomers
3   CHEBI_96557   CHEBI_96022  Diastereomers
4  CHEBI_112617  CHEBI_112586  Diastereomers
               id1           id2          label
3092  CHEBI_199087  CHEBI_203717  Diastereomers
3093  CHEBI_207142  CHEBI_215860  Diastereomers
3094   CHEBI_18809   CHEBI_18624  Diastereomers
3095  CHEBI_195839  CHEBI_195840  Diastereomers
3096  CHEBI_140655  CHEBI_138945  Diastereomers


In [8]:
# extract the ids to construct the file paths
chebi_strucs = "/media/JACK/stereomapper_structures/chebi_structures"

df_test_data['file_1'] = df_test_data['id1'].apply(lambda x: os.path.join(chebi_strucs, f"{x}.sdf"))
df_test_data['file_2'] = df_test_data['id2'].apply(lambda x: os.path.join(chebi_strucs, f"{x}.sdf"))

# extract each file path to a list
file_list_1 = df_test_data['file_1'].tolist()
file_list_2 = df_test_data['file_2'].tolist()
file_list = file_list_1 + file_list_2


def norm_chebi(x: str) -> str:
    """Return a canonical CHEBI_###### string from inputs like 'chebi:123', 'CHEBI_123', '123'."""
    s = str(x).strip()
    m = re.search(r'(\d+)$', s)  # grab trailing digits
    if not m:
        return s.upper()  # fallback: just uppercase unknowns
    return f"CHEBI_{m.group(1)}"

# 1) Canonicalize + dedupe control set
dfc = df_test_data.copy()
dfc["id1_c"] = dfc["id1"].map(norm_chebi)
dfc["id2_c"] = dfc["id2"].map(norm_chebi)
# order-invariant canonical pair (tuple sorted)
dfc["pair_key"] = dfc.apply(lambda r: tuple(sorted((r["id1_c"], r["id2_c"]))), axis=1)
# keep one row per unordered pair (optionally verify labels agree before dropping)
dfc_dedup = dfc.drop_duplicates(subset=["pair_key"]).reset_index(drop=True)

# find files that do not exist in df_test_data_enantiomers and remove their rows
# make a working copy to remove missing-file rows from
dfc_dedup_dia = dfc_dedup.copy()

for index, row in dfc_dedup.iterrows():
    file1 = os.path.join(chebi_strucs, f"{row['id1']}.sdf")
    file2 = os.path.join(chebi_strucs, f"{row['id2']}.sdf")
    if not os.path.exists(file1) or not os.path.exists(file2):
        # remove the row
        dfc_dedup_dia.drop(index, inplace=True)

control_keys = set(dfc_dedup_dia["pair_key"])

In [9]:
# compare back to original df, to permit calculation of accuracy
expanded_keys = set()
for _, row in moldf.iterrows():
    if not row["cluster_a_members"] or not row["cluster_b_members"]:
        continue
    m1s = [norm_chebi(x) for x in json.loads(row["cluster_a_members"])]
    m2s = [norm_chebi(x) for x in json.loads(row["cluster_b_members"])]
    for a in m1s:
        for b in m2s:
            if a == b:
                continue
            expanded_keys.add(tuple(sorted((a, b))))

# --- Overlap ---
tp = control_keys & expanded_keys
fp = expanded_keys - control_keys
fn = control_keys - expanded_keys

print(f"True Positives: {len(tp)}")
print(f"False Positives: {len(fp)}")
print(f"False Negatives: {len(fn)}")


NameError: name 'moldf' is not defined

In [11]:
# extract the rows that correspond to the control set only
# valid pairs are those in the control set

valid_pairs = control_keys

df_pred_filtered = []
for _, row in moldf.iterrows():
    members_1 = [norm_chebi(x) for x in json.loads(row["cluster_a_members"])]
    members_2 = [norm_chebi(x) for x in json.loads(row["cluster_b_members"])]
    for a in members_1:
        for b in members_2:
            pair = tuple(sorted((a, b)))
            if pair in valid_pairs:
                df_pred_filtered.append({"id1": a, "id2": b, **row.to_dict()})

df_pred_filtered = pd.DataFrame(df_pred_filtered)

In [12]:
df_pred_filtered

,id1,id2,cluster_a,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
0,CHEBI_224674,CHEBI_219347,1,2,"[""chebi:CHEBI:224674""]","[""chebi:CHEBI:219347""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
1,CHEBI_3829,CHEBI_81144,3,4,"[""chebi:CHEBI:3829""]","[""chebi:CHEBI:81144""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
2,CHEBI_69288,CHEBI_69289,5,6,"[""chebi:CHEBI:69288""]","[""chebi:CHEBI:69289""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3,CHEBI_117326,CHEBI_117283,7,8,"[""chebi:CHEBI:117326""]","[""chebi:CHEBI:117283""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
4,CHEBI_81326,CHEBI_81325,9,10,"[""chebi:CHEBI:81326""]","[""chebi:CHEBI:81325""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3091,CHEBI_122230,CHEBI_122370,6178,6179,"[""chebi:CHEBI:122230""]","[""chebi:CHEBI:122370""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3092,CHEBI_188987,CHEBI_188993,6180,6181,"[""chebi:CHEBI:188987""]","[""chebi:CHEBI:188993""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3093,CHEBI_217768,CHEBI_208414,6182,6183,"[""chebi:CHEBI:217768""]","[""chebi:CHEBI:208414""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3094,CHEBI_197905,CHEBI_199614,6184,6185,"[""chebi:CHEBI:197905""]","[""chebi:CHEBI:199614""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0


In [13]:
df_pred_filtered = df_pred_filtered.copy()
df_pred_filtered["pair_key"] = df_pred_filtered.apply(
    lambda r: tuple(sorted([norm_chebi(r["id1"]), norm_chebi(r["id2"])])), axis=1
)

In [14]:
df_eval = dfc_dedup.merge(
    df_pred_filtered[["pair_key", "classification", "score"]],
    on="pair_key",
    how="left",
    suffixes=("_true", "_pred")
)

In [15]:
df_eval

,id1,id2,label,file_1,file_2,id1_c,id2_c,pair_key,classification,score
0,CHEBI_122389,CHEBI_122317,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_122389,CHEBI_122317,"(CHEBI_122317, CHEBI_122389)",Diastereomers,100.0
1,CHEBI_216464,CHEBI_215732,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_216464,CHEBI_215732,"(CHEBI_215732, CHEBI_216464)",Diastereomers,100.0
2,CHEBI_120133,CHEBI_120171,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_120133,CHEBI_120171,"(CHEBI_120133, CHEBI_120171)",Diastereomers,100.0
3,CHEBI_96557,CHEBI_96022,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_96557,CHEBI_96022,"(CHEBI_96022, CHEBI_96557)",Diastereomers,100.0
4,CHEBI_112617,CHEBI_112586,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_112617,CHEBI_112586,"(CHEBI_112586, CHEBI_112617)",Parent-child,82.0
...,...,...,...,...,...,...,...,...,...,...
3092,CHEBI_199087,CHEBI_203717,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_199087,CHEBI_203717,"(CHEBI_199087, CHEBI_203717)",Diastereomers,100.0
3093,CHEBI_207142,CHEBI_215860,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_207142,CHEBI_215860,"(CHEBI_207142, CHEBI_215860)",Diastereomers,79.0
3094,CHEBI_18809,CHEBI_18624,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_18809,CHEBI_18624,"(CHEBI_18624, CHEBI_18809)",Diastereomers,84.0
3095,CHEBI_195839,CHEBI_195840,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_195839,CHEBI_195840,"(CHEBI_195839, CHEBI_195840)",Diastereomers,79.0


Final dataframe containing the predicitions made by stereomapper, only on those pairs in the control dataset. 3,097 rows as expected.

In [16]:
# only rows where we have a prediction - false negatives are those with no prediction - due to missing /unprocessed files
df_with_pred_dia = df_eval.dropna(subset=["classification"])

tp = ((df_with_pred_dia["classification"] == df_with_pred_dia["label"]).sum())
fp = ((df_with_pred_dia["classification"] != df_with_pred_dia["label"]).sum())
fn = df_eval["classification"].isna().sum()

print(f"TP: {tp}, FP: {fp}, FN: {fn}")


TP: 2901, FP: 195, FN: 1


In [23]:
# get apparent precision, recall, f1
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision +  
recall) > 0 else 0 

print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 score:  {f1:.3f}")

Precision: 0.937
Recall:    0.9997
F1 score:  0.967


Very good results indeed. Quick check into discrepancies

In [ ]:
df_with_pred_dia[df_with_pred_dia['classification'] != df_with_pred_dia['label']]#.value_counts('classification')b

,id1,id2,label,file_1,file_2,id1_c,id2_c,pair_key,classification,score
4,CHEBI_112617,CHEBI_112586,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_112617,CHEBI_112586,"(CHEBI_112586, CHEBI_112617)",Parent-child,82.0
11,CHEBI_95659,CHEBI_96178,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_95659,CHEBI_96178,"(CHEBI_95659, CHEBI_96178)",Parent-child,79.0
30,CHEBI_113375,CHEBI_96931,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_113375,CHEBI_96931,"(CHEBI_113375, CHEBI_96931)",Parent-child,85.0
46,CHEBI_95653,CHEBI_96183,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_95653,CHEBI_96183,"(CHEBI_95653, CHEBI_96183)",Parent-child,79.0
51,CHEBI_128319,CHEBI_130426,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_128319,CHEBI_130426,"(CHEBI_128319, CHEBI_130426)",Parent-child,75.0
...,...,...,...,...,...,...,...,...,...,...
2959,CHEBI_70040,CHEBI_70041,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_70040,CHEBI_70041,"(CHEBI_70040, CHEBI_70041)",Parent-child,76.0
2982,CHEBI_2318,CHEBI_5965,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_2318,CHEBI_5965,"(CHEBI_2318, CHEBI_5965)",Parent-child,76.0
2991,CHEBI_217503,CHEBI_203761,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_217503,CHEBI_203761,"(CHEBI_203761, CHEBI_217503)",Parent-child,100.0
3030,CHEBI_230185,CHEBI_31683,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_230185,CHEBI_31683,"(CHEBI_230185, CHEBI_31683)",Unclassified,NaN


In [21]:
# boolean-index the dataframe using pandas Series (use parentheses, not list literals)
df_fp_not_parent_child = df_with_pred_dia[
	(df_with_pred_dia['classification'] != 'Parent-child') &
	(df_with_pred_dia['classification'] != df_with_pred_dia['label'])
]
df_fp_not_parent_child

,id1,id2,label,file_1,file_2,id1_c,id2_c,pair_key,classification,score
324,CHEBI_196394,CHEBI_187201,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_196394,CHEBI_187201,"(CHEBI_187201, CHEBI_196394)",Unclassified,NaN
338,CHEBI_18237,CHEBI_76051,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_18237,CHEBI_76051,"(CHEBI_18237, CHEBI_76051)",Unclassified,NaN
543,CHEBI_176586,CHEBI_21547,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_176586,CHEBI_21547,"(CHEBI_176586, CHEBI_21547)",Unclassified,NaN
700,CHEBI_16449,CHEBI_132498,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_16449,CHEBI_132498,"(CHEBI_132498, CHEBI_16449)",Unclassified,NaN
701,CHEBI_66916,CHEBI_143534,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_66916,CHEBI_143534,"(CHEBI_143534, CHEBI_66916)",Unclassified,NaN
845,CHEBI_192787,CHEBI_182301,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_192787,CHEBI_182301,"(CHEBI_182301, CHEBI_192787)",Unclassified,NaN
861,CHEBI_16919,CHEBI_231575,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_16919,CHEBI_231575,"(CHEBI_16919, CHEBI_231575)",Unclassified,NaN
902,CHEBI_25017,CHEBI_231582,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_25017,CHEBI_231582,"(CHEBI_231582, CHEBI_25017)",Unclassified,NaN
934,CHEBI_9435,CHEBI_228678,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_9435,CHEBI_228678,"(CHEBI_228678, CHEBI_9435)",Unclassified,NaN
1003,CHEBI_64554,CHEBI_231586,Diastereomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_64554,CHEBI_231586,"(CHEBI_231586, CHEBI_64554)",Unclassified,NaN


6 mistakenly identified as enantiomers - double check these manually and determine true values.

- CHEBI_2699 and CHEBI_9524 => actual diastereomers
- CHEBI_50147 and CHEBI_50146 => actual diastereomers
- CHEBI_48882 and CHEBI_17486 => actual diastereomers
- CHEBI_15711 and CHEBI_15711 => actual diastereomers
- CHEBI_85535 and CHEBI_85536 => enantiomers, but wilcard atoms - shouldnt be here
- CHEBI_233817 and CHEBI_68828 => enantiomers

Therefore, stereomapper incorrectly classifies 4 structures as enantiomers, which are actually diastereomers. Issues likely lie with inchi software, as inchi strings for each pair have the same stereochemical assignments. An obvious limitation of stereomapper is it's reliance on the accuracy of the InChI software.

In terms of the structures containing wildcard atoms, perhaps a more stringent test is required than reliance on rdkit code, maybe a formula check would be sufficient? or a direct check of the molfile?

Lastly, CHEBI_233817 and CHEBI_68828 are indeed enantiomers upon manual review.